## 목표
- 모델 평가 기준(metrics)과 EarlyStopping monitor 기준 f1_score로 해서 모델의 학습 상황 점검

In [9]:
# import librosa
# import librosa.display
# import IPython.display as ipd
# from IPython.display import Audio

from random import randint
import numpy as np
import pandas as pd
# import pydub #오디오 파일을 다루는 라이브러리 : 변환 조작 재생 분석 등
# import gTTS # 텍스트를 음성으로 변환하는 라이브러리. 나는 당장 쓸 일 없을 듯?

# from tqdm import tqdm
# import np_utils
# import matplotlib.pyplot as plt
# import seaborn as sns

# import winsound as sd
# import glob
# import os
# import json
# import shutil
# import sys
# import logging
# import unicodedata
# from shutil import copyfile
# import warnings
# if not sys.warnoptions:
#     warnings.simplefilter("ignore")
# warnings.filterwarnings("ignore", category=DeprecationWarning) 


from sklearn.preprocessing import LabelEncoder,StandardScaler, OneHotEncoder, scale, MinMaxScaler
# from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

import joblib
from joblib import dump, load
from xgboost import XGBClassifier 

import keras
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping
from keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv1D, MaxPooling1D, Flatten, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
# from keras.preprocessing.sequence import pad_sequences
# from keras.utils import to_categorical

import tensorflow as tf
from sentence_transformers import SentenceTransformer


# from transformers import PreTrainedTokenizer




In [2]:
train_csv = pd.read_csv('e:/Data2/csv/train.csv')
valid_csv = pd.read_csv('e:/Data2/csv/valid.csv')
X = np.load('./features5.npy')
y = train_csv.iloc[:,-7:]

### 필요한 함수들 만들기

In [31]:
def beepError():
    fr = 1000    # range : 37 ~ 32767
    du = 2000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
def beepSuccess():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)

In [4]:
# 임베딩 함수 설정
class text_embedding():
    def __init__(self, model_name):
        self.model_name = model_name

    def fit(self, X, y=None):
        return self
  
    def transform(self, X,raw_label):
        embedding_model = SentenceTransformer(self.model_name)
        embedding_vec = embedding_model.encode(raw_label)
        X_val = np.concatenate((X, embedding_vec), axis = 1)
        return X_val

In [58]:
def cnn_model(data):
    model=Sequential()
    model.add(InputLayer(shape=(data.shape[1],data.shape[2])))
    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
    model.add(Dropout(0.2))

    model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Flatten())
    model.add(Dense(units=32, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(units=7, activation='softmax'))
    model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy',f1_metric])

    return model

In [75]:
# f1 스코어를 쓸 수 있게 만드는 함수
# f1 스코어의 결과 타입은 tensor이므로 이걸 변환해 주는 작업 필요.

def f1_metric(y_true, y_pred):
    y_true = tf.argmax(y_true, axis=1)
    y_pred = tf.argmax(y_pred, axis=1)
    return tf.py_function(lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted'), 
                          [y_true, y_pred], Tout=tf.float64)

In [52]:
# 기본적인 사전 설정
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=5, min_lr=0.0000001) #learning rate 조절 
modelpath = './model/model_{epoch:02d}-{val_accuracy:.4f}.keras'
mcp = ModelCheckpoint(
    modelpath,     #저장할 모델의 경로
  monitor = 'val_f1_metric', #val_acc를 기준으로 전보다 모델이 나아지는 걸 확인
  save_best_only = True,    #나아진 결과만 저장
#     save_weights_only=True , #이걸 써 줘야 weights.h5로 저장 가능하다.
  verbose = 1               #과정을 출력
)

#전보다 나아지지 않으면 학습중단

es = EarlyStopping(
    monitor = 'val_f1',
    patience = 5      # 전보다 나아지지 않아도 실행할 횟수
)

In [77]:
pre_trained_models = [
#     'sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens',
# 'sentence-transformers/multi-qa-distilbert-cos-v1',
'jhgan/ko-sroberta-multitask',
# 'all-distilroberta-v1',
# 'jhgan/ko-sbert-multitask',
# 'all-MiniLM-L12-v2', 'jhgan/ko-sroberta-sts'
]
y_np = np.array(y)
scaler = StandardScaler()

for i in pre_trained_models:
#     train_txt_embed = text_embedding(model_name = i)
#     X_train_embed = train_txt_embed.transform(X,train_csv['발화문'])
   
    X_train, X_test, y_train, y_test = train_test_split(X_train_embed,y_np, random_state=0,  test_size = 0.3, stratify = y_np)

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_train_scaled = np.expand_dims(X_train_scaled,axis =1)
    X_test_scaled = np.expand_dims(X_test_scaled,axis =1)


    model = cnn_model(X_train_scaled)
    history=model.fit(
        X_train_scaled, 
        y_train,batch_size=40, 
        epochs=50, 
        validation_data=(X_test_scaled, y_test), 
        callbacks=[rlrp,mcp]
    )
    test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)
    print("Pre-trained Model: ", i)
    print("Test Accuracy: ",test_acc)
    
#결과 
#'sentence-transformers/multi-qa-distilbert-cos-v1'로 임베딩한 게 가장 좋다.(약 81.96%)
# 'jhgan/ko-sbert-multitask'과 'jhgan/ko-sroberta-sts'도 나름대로 준수
# 모델명 : model_06-0.8196.keras

Epoch 1/50


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 7)